# Problem

Is to identify products at risk of backorder before the event occurs so that business has time to react. 

## What is a Backorder?
Backorders are products that are temporarily out of stock, but a customer is permitted to place an order against future inventory. 
A backorder generally indicates that customer demand for a product or service exceeds a company’s capacity to supply it. Back orders are both good and bad. Strong demand can drive back orders, but so can suboptimal planning. 

## Data description

Data file contains the historical data for the 8 weeks prior to the week we are trying to predict. The data was taken as weekly snapshots at the start of each week. Columns are defined as follows:

    sku - Random ID for the product

    national_inv - Current inventory level for the part

    lead_time - Transit time for product (if available)

    in_transit_qty - Amount of product in transit from source

    forecast_3_month - Forecast sales for the next 3 months

    forecast_6_month - Forecast sales for the next 6 months

    forecast_9_month - Forecast sales for the next 9 months

    sales_1_month - Sales quantity for the prior 1 month time period

    sales_3_month - Sales quantity for the prior 3 month time period

    sales_6_month - Sales quantity for the prior 6 month time period

    sales_9_month - Sales quantity for the prior 9 month time period

    min_bank - Minimum recommend amount to stock

    potential_issue - Source issue for part identified

    pieces_past_due - Parts overdue from source

    perf_6_month_avg - Source performance for prior 6 month period

    perf_12_month_avg - Source performance for prior 12 month period

    local_bo_qty - Amount of stock orders overdue

    deck_risk - Part risk flag

    oe_constraint - Part risk flag

    ppap_risk - Part risk flag

    stop_auto_buy - Part risk flag

    rev_stop - Part risk flag

    went_on_backorder - Product actually went on backorder. This is the target value.
    
         Yes or 1 : Product backordered

         No or 0  : Product not backordered

# Loading the required libraries

In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv("BackOrders.csv",header=0)

In [3]:
data.head()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,1888279,117,NaN,0,0,0,0,0,0,15,...,0,-99.00,-99.00,0,No,No,Yes,Yes,No,No
1,1870557,7,2.0,0,0,0,0,0,0,0,...,0,0.50,0.28,0,Yes,No,No,Yes,No,No
2,1475481,258,15.0,10,10,77,184,46,132,256,...,0,0.54,0.70,0,No,No,No,Yes,No,No
3,1758220,46,2.0,0,0,0,0,1,2,6,...,0,0.75,0.90,0,Yes,No,No,Yes,No,No
4,1360312,2,2.0,0,4,6,10,2,2,5,...,0,0.97,0.92,0,No,No,No,Yes,No,No


In [4]:
def evaluate_model(act, pred):
    from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
    print("Confusion Matrix \n", confusion_matrix(act, pred))
    print("Accurcay : ", accuracy_score(act, pred))
    print("Recall   : ", recall_score(act, pred))
    print("Precision: ", precision_score(act, pred))    

In [5]:
data.dtypes

sku                    int64
national_inv           int64
lead_time            float64
in_transit_qty         int64
forecast_3_month       int64
forecast_6_month       int64
forecast_9_month       int64
sales_1_month          int64
sales_3_month          int64
sales_6_month          int64
sales_9_month          int64
min_bank               int64
potential_issue       object
pieces_past_due        int64
perf_6_month_avg     float64
perf_12_month_avg    float64
local_bo_qty           int64
deck_risk             object
oe_constraint         object
ppap_risk             object
stop_auto_buy         object
rev_stop              object
went_on_backorder     object
dtype: object

In [6]:
for col in ['sku', 'potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 
            'stop_auto_buy', 'rev_stop', 'went_on_backorder']:
    data[col] = data[col].astype('category')

In [7]:
data.dtypes

sku                  category
national_inv            int64
lead_time             float64
in_transit_qty          int64
forecast_3_month        int64
forecast_6_month        int64
forecast_9_month        int64
sales_1_month           int64
sales_3_month           int64
sales_6_month           int64
sales_9_month           int64
min_bank                int64
potential_issue      category
pieces_past_due         int64
perf_6_month_avg      float64
perf_12_month_avg     float64
local_bo_qty            int64
deck_risk            category
oe_constraint        category
ppap_risk            category
stop_auto_buy        category
rev_stop             category
went_on_backorder    category
dtype: object

In [8]:
data.isnull().sum()

sku                     0
national_inv            0
lead_time            3403
in_transit_qty          0
forecast_3_month        0
forecast_6_month        0
forecast_9_month        0
sales_1_month           0
sales_3_month           0
sales_6_month           0
sales_9_month           0
min_bank                0
potential_issue         0
pieces_past_due         0
perf_6_month_avg        0
perf_12_month_avg       0
local_bo_qty            0
deck_risk               0
oe_constraint           0
ppap_risk               0
stop_auto_buy           0
rev_stop                0
went_on_backorder       0
dtype: int64

In [9]:
data = data.dropna(axis=0)

In [11]:
data.isnull().sum()

sku                  0
national_inv         0
lead_time            0
in_transit_qty       0
forecast_3_month     0
forecast_6_month     0
forecast_9_month     0
sales_1_month        0
sales_3_month        0
sales_6_month        0
sales_9_month        0
min_bank             0
potential_issue      0
pieces_past_due      0
perf_6_month_avg     0
perf_12_month_avg    0
local_bo_qty         0
deck_risk            0
oe_constraint        0
ppap_risk            0
stop_auto_buy        0
rev_stop             0
went_on_backorder    0
dtype: int64

In [12]:
data['went_on_backorder'].value_counts(normalize=True)*100

No     81.148386
Yes    18.851614
Name: went_on_backorder, dtype: float64

In [13]:
data.drop('sku', axis=1, inplace=True)

/usr/share/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
data.columns

Index(['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month',
       'forecast_6_month', 'forecast_9_month', 'sales_1_month',
       'sales_3_month', 'sales_6_month', 'sales_9_month', 'min_bank',
       'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
       'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder'],
      dtype='object')

In [15]:
X = data.drop('went_on_backorder', axis=1)
y = data['went_on_backorder']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=123) 

In [18]:
y_train.value_counts(True)

No     0.811859
Yes    0.188141
Name: went_on_backorder, dtype: float64

In [17]:
y_test.value_counts(True)

No     0.81061
Yes    0.18939
Name: went_on_backorder, dtype: float64

## Convert categorical to numerical

In [21]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y_train)

y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [23]:
pd.value_counts(y_train)/y_train.size * 100

0    81.185858
1    18.814142
dtype: float64

In [26]:
cat_attr = X_train.select_dtypes(include=['category']).columns



In [27]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(drop = 'first')
enc.fit(X_train[cat_attr])

X_train_ohe=enc.transform(X_train[cat_attr]).toarray()
X_test_ohe=enc.transform(X_test[cat_attr]).toarray()

## standardize numerical variables

In [28]:
num_attr = X_train.select_dtypes(include=['float64', 'int64']).columns

In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train[num_attr])

X_train_std = scaler.transform(X_train[num_attr])
X_test_std = scaler.transform(X_test[num_attr])

## Concatenate

In [32]:
X_train_con = np.concatenate([X_train_std, X_train_ohe], axis=1)
X_test_con = np.concatenate([X_test_std, X_test_ohe], axis=1)

## Model building

In [33]:
from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier()
clf1.fit(X_train_con, y_train)

train_pred = clf1.predict(X_train_con)
test_pred = clf1.predict(X_test_con)

In [34]:
print("--Train--")
evaluate_model(y_train, train_pred)
print("--Test--")
evaluate_model(y_test, test_pred)

--Train--
Confusion Matrix 
 [[33022    45]
 [   44  7619]]
Accurcay :  0.9978148784679597
Recall   :  0.9942581234503458
Precision:  0.9941283924843424
--Test--
Confusion Matrix 
 [[13594   556]
 [  587  2719]]
Accurcay :  0.9345210815765352
Recall   :  0.822444041137326
Precision:  0.8302290076335878


importances = clf1.feature_importances_
print(importances)

indices = np.argsort(importances)[::-1]
print(indices)

ind_attr_names = np.concatenate((num_attr, cat_attr), axis=0)
pd.DataFrame([ind_attr_names[indices], np.sort(importances)[::-1]])

## Upsampling

In [35]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=123)
X_train_sm, y_train_sm = smote.fit_resample(X_train_con, y_train)

In [ ]:
np.unique(y_train, return_counts= True)
np.unique(y_train_sm, return_counts= True)

In [38]:
#from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier()
clf2.fit(X_train_sm, y_train_sm)

train_pred = clf2.predict(X_train_sm)
test_pred = clf2.predict(X_test_con)

In [40]:
print("--Train--")
evaluate_model(y_train_sm, train_pred)
print("--Test--")
evaluate_model(y_test, test_pred)

--Train--
Confusion Matrix 
 [[33008    59]
 [   48 33019]]
Accurcay :  0.9983820727613633
Recall   :  0.9985484017298213
Precision:  0.9982163371425117
--Test--
Confusion Matrix 
 [[13313   837]
 [  407  2899]]
Accurcay :  0.9287351054078826
Recall   :  0.8768905021173624
Precision:  0.7759635974304069


## Hyper params

In [42]:
param_grid = {"n_estimators" : [50, 100],
              "max_depth" : [1,5],
              "max_features" : [3, 5],
              "min_samples_leaf" : [1, 2, 4]}

In [44]:
clf3 = RandomForestClassifier()
from sklearn.model_selection import GridSearchCV
clf_grid = GridSearchCV(clf3, param_grid, cv=2)

clf_grid.fit(X_train_sm, y_train_sm)

train_pred = clf_grid.predict(X_train_sm)
test_pred = clf_grid.predict(X_test_con)

In [45]:
print("--Train--")
evaluate_model(y_train_sm, train_pred)
print("--Test--")
evaluate_model(y_test, test_pred)

--Train--
Confusion Matrix 
 [[27826  5241]
 [ 3060 30007]]
Accurcay :  0.8744821120754831
Recall   :  0.9074606102761061
Precision:  0.8513107126645484
--Test--
Confusion Matrix 
 [[11848  2302]
 [  481  2825]]
Accurcay :  0.840570577451879
Recall   :  0.8545069570477919
Precision:  0.5510044860542227
